
本程式碼請搭配文章「[在 Colab 上利用 Yolov3 框架和自有標住資料來訓練自己的物件辨識系統](http://bit.ly/2OAfpq0)」 一起服用。

其實原意是利用圖片辨識手遊的畫面，來幫我自動玩手遊練功。在找資料的時候，很多人都推薦 Yolov3，優點是非常快速，剛好利用在畫面變動巨大的手遊上。但使用遊戲畫面，可能只有我自己看得懂。所以這篇文章我用最普遍的貓和狗當作範例。能訓練貓和狗就能訓練自己其他的資料了。

# 這篇文章會教你
在此範例中，我們將會訓練一個能偵測圖片中貓和狗的模型。你可以自由替換資料集，偵測自訂的物體。

除此之外，還有：
* 利用 Colab 128G RAM GPU 來訓練你的 Yolo3 模型
* 掛載 Google Drive 檔案到 Colab 檔案系統中
* 將 PASCAL VOC 標籤格式轉換成 Yolo 用的標籤格式
* 產生 Yolo 訓練需要的 cfg 設定檔案
* 將訓練後的 weight 檔案同步至 Google Drive 中，避免遺失
* 如何利用 weight 檔案來辨識圖片中的內容

# 事前準備

在開始前，你需要先編譯好 Darknet 執行檔案。你可以參考我的另外一篇文章「[如何在 Colab 安裝 Darknet 框架訓練 YOLO v3 物件辨識並且最佳化 Colab 的訓練流程](http://bit.ly/33XjcEu)」，文章中會將編譯好的 Darknet 執行檔案放到 Google Drive目錄下。本文章將會利用這個執行檔案進行訓練。




---




## 將 Google Drive 掛載到 Colab 目錄下

掛載 Google Drive 的好處是不用每次都手動上傳或下載檔案，而且還能讓訓練好的模型檔案自動保存到 Google Drive。這樣就不會因為 Colab 中斷後就必須從頭訓練。

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'
!git clone https://github.com/AlexeyAB/darknet/

Cloning into 'darknet'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 12558 (delta 11), reused 11 (delta 3), pack-reused 12533
Receiving objects: 100% (12558/12558), 11.66 MiB | 15.06 MiB/s, done.
Resolving deltas: 100% (8523/8523), done.


In [3]:
!apt install gcc-5 g++-5 -y
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc 
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  cpp-5 gcc-5-base libasan2 libgcc-5-dev libisl15 libmpx0 libstdc++-5-dev
Suggested packages:
  gcc-5-locales g++-5-multilib gcc-5-doc libstdc++6-5-dbg gcc-5-multilib
  libgcc1-dbg libgomp1-dbg libitm1-dbg libatomic1-dbg libasan2-dbg
  liblsan0-dbg libtsan0-dbg libubsan0-dbg libcilkrts5-dbg libmpx0-dbg
  libquadmath0-dbg libstdc++-5-doc
The following NEW packages will be installed:
  cpp-5 g++-5 gcc-5 gcc-5-base libasan2 libgcc-5-dev libisl15 libmpx0
  libstdc++-5-dev
0 upgraded, 9 newly installed, 0 to remove and 25 not upgraded.
Need to get 29.1 MB of archives.
After this operation, 100 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 gcc-5-base amd64 

In [4]:
%cd darknet
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:909:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -c ./src/http_stream.cpp -o obj/http_stream.o
In file included 

In [0]:
# !wget https://pjreddie.com/media/files/yolov3.weights
# !chmod a+x ./darknet

In [0]:
# from google.colab import files
# uploaded = files.upload()

Saving Kangaroo.mp4 to Kangaroo.mp4


In [5]:
!pwd

/content/darknet


In [0]:
!cp /content/gdrive/My\ Drive/train_yolo_with_custom_dataset_on_colab_101/weights/yolov3_last.weights /content/darknet

In [0]:
!cp /content/gdrive/My\ Drive/train_yolo_with_custom_dataset_on_colab_101/cfg/obj.data /content/darknet

In [0]:
!cp /content/gdrive/My\ Drive/train_yolo_with_custom_dataset_on_colab_101/cfg/yolov3.cfg /content/darknet

In [9]:
%cd ..

/content


In [0]:
!mkdir app

In [11]:
%cd app
!mkdir cfg
!cp /content/gdrive/My\ Drive/train_yolo_with_custom_dataset_on_colab_101/cfg/obj.names /content/app/cfg

/content/app


In [12]:
%cd ..

/content


In [13]:
%cd darknet

/content/darknet


In [0]:
videopth = "/content/gdrive/My\ Drive/Final_Project/train_yolo_with_custom_dataset_on_colab_101"

In [15]:
!./darknet detector demo obj.data yolov3.cfg yolov3_last.weights -dont_show {videopth}/Kangaroo.mp4 -out_filename Kangaroo.avi 
# !./darknet detector demo obj.data yolov3.cfg yolov3_last.weights -dont_show {videopth}/Kangaroo.mp4 -i 0 -ext_output Kangaroo.avi

Streaming output truncated to the last 5000 lines.


FPS:40.1 	 AVG_FPS:40.5

 cvWriteFrame 
Objects:


FPS:40.1 	 AVG_FPS:40.5

 cvWriteFrame 
Objects:

kangaroo: 31% 

FPS:40.1 	 AVG_FPS:40.5

 cvWriteFrame 
Objects:

kangaroo: 88% 
kangaroo: 43% 

FPS:40.2 	 AVG_FPS:40.5

 cvWriteFrame 
Objects:

kangaroo: 87% 
kangaroo: 43% 

FPS:40.1 	 AVG_FPS:40.5

 cvWriteFrame 
Objects:

kangaroo: 86% 
kangaroo: 58% 

FPS:40.2 	 AVG_FPS:40.5

 cvWriteFrame 
Objects:

kangaroo: 64% 
kangaroo: 39% 

FPS:39.2 	 AVG_FPS:40.5

 cvWriteFrame 
Objects:

kangaroo: 48% 

FPS:39.1 	 AVG_FPS:40.5

 cvWriteFrame 
Objects:

kangaroo: 29% 

FPS:39.1 	 AVG_FPS:40.5

 cvWriteFrame 
Objects:

kangaroo: 29% 

FPS:38.8 	 AVG_FPS:40.5

 cvWriteFrame 
Objects:

kangaroo: 70% 

FPS:39.0 	 AVG_FPS:40.5

 cvWriteFrame 
Objects:

kangaroo: 33% 

FPS:39.1 	 AVG_FPS:40.5

 cvWriteFrame 
Objects:

kangaroo: 83% 
kangaroo: 48% 

FPS:39.3 	 AVG_FPS:40.5

 cvWriteFrame 
Objects:

kangaroo: 97% 
kangaroo: 90% 
kangaroo: 59% 


In [0]:
from google.colab import files
downloaded = files.download('Kangaroo.avi')

In [16]:
!./darknet detector demo obj.data yolov3.cfg yolov3_last.weights -dont_show {videopth}/Raccoon.mp4 -out_filename Raccoon.avi

Streaming output truncated to the last 5000 lines.
 cvWriteFrame 
Objects:

raccoon: 100% 

FPS:39.3 	 AVG_FPS:37.6

 cvWriteFrame 
Objects:

raccoon: 99% 

FPS:39.1 	 AVG_FPS:37.6

 cvWriteFrame 
Objects:

raccoon: 85% 

FPS:38.4 	 AVG_FPS:37.6

 cvWriteFrame 
Objects:

raccoon: 97% 

FPS:38.5 	 AVG_FPS:37.6

 cvWriteFrame 
Objects:

raccoon: 99% 

FPS:38.3 	 AVG_FPS:37.6

 cvWriteFrame 
Objects:

raccoon: 100% 

FPS:38.1 	 AVG_FPS:37.6

 cvWriteFrame 
Objects:

raccoon: 99% 

FPS:37.7 	 AVG_FPS:37.6

 cvWriteFrame 
Objects:

raccoon: 100% 

FPS:38.0 	 AVG_FPS:37.6

 cvWriteFrame 
Objects:

raccoon: 99% 

FPS:38.3 	 AVG_FPS:37.6

 cvWriteFrame 
Objects:

raccoon: 99% 

FPS:38.0 	 AVG_FPS:37.6

 cvWriteFrame 
Objects:

raccoon: 99% 

FPS:38.3 	 AVG_FPS:37.6

 cvWriteFrame 
Objects:

raccoon: 99% 

FPS:38.5 	 AVG_FPS:37.6

 cvWriteFrame 
Objects:

raccoon: 99% 

FPS:38.5 	 AVG_FPS:37.6

 cvWriteFrame 
Objects:

raccoon: 100% 

FPS:39.5 	 AVG_FPS:37.6

 cvWriteFrame 
Objects:

raccoon: 9

In [0]:
downloaded2 = files.download('Raccoon.avi')